# <font color="purple">Adversarial Training of Residual Network</font>

Definition of WideResNet and Architecture:

##  <center>The Result of Residual Network </center>

***Road Map***
* Data Preprocessing
* Model Cross Validation Results
* Evaluate the GridSearchCV Results
* Model Training and Learning Curves
* Model Adversarial Training Approach.


**Libraries**

In [3]:
import sys
sys.path.insert(1,'/home/sefika/AE_Parseval_Network/src/cleverhans/future/tf2/attacks')
import cleverhans


In [4]:
!pip install -qq -e git+http://github.com/tensorflow/cleverhans.git#egg=cleverhans
import sys
sys.path.append('/content/src/cleverhans')
import cleverhans

In [5]:
import tensorflow as tf
from cleverhans.future.tf2.attacks import fast_gradient_method
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import  KFold
import gzip
import pickle
import numpy as np
from parsevalnet import ParsevalNetwork
import warnings
warnings.filterwarnings("ignore")

print("\nTensorflow Version: " + tf.__version__)
# utility functions
from preprocessing import preprocessing_data
# Define configuration parameters
from _utility import lrate
from training import train
from adversarial_training import  AdversarialTraining


Tensorflow Version: 2.3.0


## <font color="green"> Data Preprocessing </font>


* Read Data from File

In [6]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()

* Call data preprocessing function

In [7]:
X, y = preprocessing_data(data)
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size = 0.1)
x_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.1)

## <font color="green"> Utilize Functions </font>

* Flipping the image using data augmentation technique

In [8]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

### <font color = "green">Some Parameters Regarding Adversarial Examples</font>

In [9]:
# predefined epsilon values
epsilon_list = [0.003,0.005,0.01,0.02]

## <font color="purple"> 1.) Baseline of the Model</font>

In [10]:
EPOCHS = 50
BS = 64
init = (32, 32,1)
sgd = SGD(lr=0.1, momentum=0.9)
parameter = {'epochs': EPOCHS, 'batch_size': BS, 'optimizer': sgd}

In [11]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
callbacks_list = [lrate, es]
parseval = ParsevalNetwork(init,0.0001, 0.9, nb_classes=4, N=2, k=1, dropout=0.0)

In [ ]:
result_df = train(parseval, X_train, Y_train, X_test, y_test, EPOCHS, BS, sgd, generator, callbacks_list, epsilon_list, model_name="Parseval")

In [13]:
result_df["clean_mean"] = np.sum(result_df['acc_clean'])/10.0
result_df["0.003_mean"] = np.sum(result_df['0.003_acc'])/10.0
result_df["0.005_mean"] = np.sum(result_df['0.005_acc'])/10.0
result_df["0.02_mean"] = np.sum(result_df['0.02_acc'])/10.0
result_df["0.01_mean"] = np.sum(result_df['0.01_acc'])/10.0

In [14]:
result_df.head(1)

,loss_clean,acc_clean,0.003_loss,0.003_acc,0.005_loss,0.005_acc,0.02_acc,0.02_loss,0.01_acc,0.01_clean,0.01_loss,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.849743,0.684119,0.966569,0.647469,1.049518,0.609075,0.525305,1.269353,0.406632,NaN,0.406632,0.703141,0.657941,0.626003,0.555323,0.424607


## <font color="purple">2.) Adversarial Training on Baseline Model</font>

In [15]:
adversarial_training =  AdversarialTraining(parameter)
result_adv_df = adversarial_training.train(parseval, X_train, Y_train, X_test, y_test, epsilon_list, callbacks_list,model_name="parseval")

72/72 [==============================] - 2s 24ms/step - loss: 0.6590 - acc: 0.7665 - val_loss: 0.7585 - val_acc: 0.7184
Epoch 48/50
72/72 [==============================] - 2s 24ms/step - loss: 0.6597 - acc: 0.7674 - val_loss: 0.7659 - val_acc: 0.7223
Epoch 49/50
72/72 [==============================] - 2s 23ms/step - loss: 0.6706 - acc: 0.7624 - val_loss: 0.7801 - val_acc: 0.7204
Epoch 50/50
18/18 [==============================] - 0s 4ms/step - loss: 0.9315 - acc: 0.6876
epsilon: 0.003 and test evaluation : 0.931526243686676, 0.687609076499939
SNR: 50.23597717285156
18/18 [==============================] - 0s 4ms/step - loss: 1.0167 - acc: 0.6597
epsilon: 0.005 and test evaluation : 1.0167371034622192, 0.6596858501434326
SNR: 45.798683166503906
18/18 [==============================] - 0s 5ms/step - loss: 1.2370 - acc: 0.5759
epsilon: 0.01 and test evaluation : 1.2369736433029175, 0.5759162306785583
SNR: 39.77808713912964
18/18 [==============================] - 0s 5ms/step - loss: 1.

In [16]:
result_adv_df

,loss_clean,acc_clean,0.003_loss,0.003_acc,0.005_loss,0.005_acc,0.02_acc,0.02_loss,0.01_acc,0.01_loss
0,0.882291,0.698080,1.029637,0.638743,1.136114,0.616056,0.521815,1.423629,0.387435,2.016575
1,0.813650,0.706806,0.928804,0.654450,1.008725,0.626527,0.546248,1.213325,0.427574,1.621001
2,0.814444,0.720768,0.930393,0.685864,1.011546,0.645724,0.568935,1.222115,0.448517,1.648703
3,0.828098,0.703316,0.958277,0.659686,1.049816,0.636998,0.568935,1.288000,0.422339,1.771215
4,0.788306,0.724258,0.920664,0.673647,1.014427,0.643979,0.588133,1.259215,0.432810,1.747693
5,0.811801,0.717277,0.931526,0.687609,1.016737,0.659686,0.575916,1.236974,0.432810,1.668866
6,0.805691,0.703316,0.916099,0.656195,0.992109,0.626527,0.544503,1.187817,0.434555,1.583908
7,0.859200,0.682373,0.972273,0.649215,1.052511,0.619546,0.546248,1.262573,0.422339,1.683952
8,0.837950,0.712042,0.963146,0.666667,1.051290,0.633508,0.561955,1.279087,0.431065,1.741798
9,0.819253,0.694590,0.934038,0.659686,1.013032,0.642234,0.584642,1.216251,0.469459,1.622038


### <font color="purple"> Results of Adversarial Training</font>

In [18]:
result_adv_df["clean_mean"] = np.sum(result_adv_df['acc_clean'])/10.0
result_adv_df["0.003_mean"] = np.sum(result_adv_df['0.003_acc'])/10.0
result_adv_df["0.005_mean"] = np.sum(result_adv_df['0.005_acc'])/10.0
result_adv_df["0.02_mean"] = np.sum(result_adv_df['0.02_acc'])/10.0
result_adv_df["0.01_mean"] = np.sum(result_adv_df['0.01_acc'])/10.0

In [19]:
column = ["clean_mean","0.003_mean","0.005_mean","0.02_mean","0.01_mean"]
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.706283,0.663176,0.635079,0.560733,0.43089


# <font color="blue">Compare Non-Adversarial Training with Adversarial Training</font>

In [20]:
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.706283,0.663176,0.635079,0.560733,0.43089


In [21]:
result_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.703141,0.657941,0.626003,0.555323,0.424607


# <font color="blue">Conclusion</font>